In [ ]:
def get_popular_movies_by_genre(genre, min_num_ratings, num_recs):
    # Filter movies by genre
    genre_movies_df = movies_df[movies_df['genres'].apply(lambda x: genre in x.split('|'))]

    # Merge with ratings data
    merged_df = pd.merge(genre_movies_df, ratings_df, on='movieId')

    # Group by movie and calculate average rating
    avg_ratings_df = merged_df.groupby('movieId')['rating'].mean().reset_index()

    # Group by movie and calculate number of ratings
    num_ratings_df = merged_df.groupby('movieId')['rating'].count().reset_index()

    # Merge average ratings and number of ratings dataframes
    merged_avg_num_df = pd.merge(avg_ratings_df, num_ratings_df, on='movieId', suffixes=('_avg', '_num'))

    # Filter by minimum number of ratings threshold
    filtered_df = merged_avg_num_df[merged_avg_num_df['rating_num'] >= min_num_ratings]

    # Sort by rating in descending order and get top N movies
    top_movies_df = filtered_df.sort_values(by='rating_avg', ascending=False).head(num_recs)

    # Merge with movies dataframe to get movie names
    top_movies_with_names_df = pd.merge(top_movies_df, movies_df, on='movieId')

    # Return the top movies with names
    return top_movies_with_names_df[['title', 'rating_avg']]


genre = input("Enter the gevere: ")
min_num_ratings = int(input("Enter minimum number of ratings: "))
num_recs = int(input("Enter number of movies you want: "))

popular_movies_df = get_popular_movies_by_genre(genre, min_num_ratings, num_recs)
print(popular_movies_df)

Enter the gevere: Action
Enter minimum number of ratings: 50
Enter number of movies you want: 5
                                       title  rating_avg
0                          Fight Club (1999)    4.272936
1                    Dark Knight, The (2008)    4.238255
2                 Princess Bride, The (1987)    4.232394
3  Star Wars: Episode IV - A New Hope (1977)    4.231076
4                      Apocalypse Now (1979)    4.219626


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the movies data
movies = pd.read_csv('/content/movies.csv')

# Define a function to get similar movies by genre
def get_similar_movies(movie_title, num_recs):
    # Create a TF-IDF vectorizer for movie genres
    vectorizer = TfidfVectorizer(stop_words='english')
    genre_vectors = vectorizer.fit_transform(movies['genres'])

    # Get the index of the input movie
    movie_idx = movies[movies['title'] == movie_title].index[0]

    # Calculate cosine similarity with all other movies
    similarities = cosine_similarity(genre_vectors[movie_idx], genre_vectors).flatten()

    # Get the indices of the top N similar movies
    similar_indices = similarities.argsort()[-num_recs-1:-1][::-1]

    # Return the titles of the similar movies
    return movies.iloc[similar_indices]['title'].tolist()

movie_title = input("Enter name: ")
num_recs = int(input("Enter number of movies you want: "))

similar_movies = get_similar_movies(movie_title, num_recs)
print(similar_movies)

Enter name: Toy Story (1995)
Enter number of movies you want: 5
["Emperor's New Groove, The (2000)", 'Antz (1998)', 'Adventures of Rocky and Bullwinkle, The (2000)', 'Wild, The (2006)', 'Asterix and the Vikings (Astérix et les Vikings) (2006)']
